In [2]:
import openai
from pymongo import MongoClient
import chromadb
from chromadb.config import Settings
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import os
import openai

from langgraph.graph import StateGraph, END
from langgraph.graph.message import MessageGraph
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage
from langchain_core.runnables import Runnable, RunnableLambda

In [3]:
# Conexão com o ChromaClient Persistente
chroma_client = chromadb.PersistentClient(path="./chroma_db")



In [4]:
# Chave da API da openai
openai_api_key = os.getenv("OPENAI_API_KEY")

# Conectar ao MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["rag_db"]
mongo_chunks = mongo_db["chunks"]

collection = chroma_client.get_collection("rag_embeddings")

# Inicializar embeddings
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key) #text-embedding-ada-002

/tmp/ipykernel_139203/571716397.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key) #text-embedding-ada-002


In [5]:


# Definindo os estados comppartilhados
class ChatState(TypedDict):
    input: str
    query: str
    context: str
    answer: str
    validation: str

# Agente de busca
def retrieve_context(state: ChatState) -> ChatState:
    query = state["input"]  # <- recebe a entrada inicial do usuário
    query_embedding = embedding_model.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding], 
        n_results=5
    )

    chunk_ids = []
    for result in results["metadatas"][0]:
        chunk_ids.append(result["source_file"] + "_chunk_" + str(result["chunk_index"]))

    chunks_text = []
    for chunk_id in chunk_ids:
        chunk = mongo_chunks.find_one({"chunk_id": chunk_id})
        if chunk:
            chunks_text.append(chunk["chunk_text"])

    return {
        **state,
        "query": query,                  # necessário para os próximos nós
        "context": "\n\n".join(chunks_text)
    }

# Agente que comunica com a openAI
def generate_answer(state: ChatState) -> ChatState:
    client = openai.OpenAI(api_key=openai_api_key)
    
    prompt = f"""
Você é um assistente especialista em licenciamento ambiental, com foco em auxiliar na elaboração de documentos EIA e RIMA. Use exclusivamente o contexto a seguir para responder de forma precisa e técnica.

Contexto:
{state['context']}

Pergunta:
{state['query']}
    """
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente técnico ambiental especializado em EIA e RIMA."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=1000
    )
    
    return {
        **state,
        "answer": response.choices[0].message.content.strip()
    }

# Agente que valida a resṕosta obtida
def AnswerValidatorAgent(state: ChatState) -> ChatState:
    client = openai.OpenAI(api_key=openai_api_key)

    answer = state["answer"]
    context = state["context"]
    query = state["query"]

    validation_prompt = f"""
Você é um validador técnico. Avalie a resposta de um assistente que tenta responder perguntas sobre EIA e RIMA com base em um contexto técnico.

Contexto usado:
{context}

Pergunta feita:
{query}

Resposta do assistente:
{answer}

Agora avalie a resposta com base nos critérios abaixo:
1. A resposta está diretamente relacionada ao contexto?
2. Há alguma afirmação vaga, genérica ou sem apoio no contexto?
3. A linguagem está tecnicamente correta e clara?

Se a resposta estiver satisfatória, diga "VALIDADO".
Se não, explique o problema encontrado.
"""

    validation_response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um validador técnico de respostas geradas por IA."},
            {"role": "user", "content": validation_prompt}
        ],
        temperature=0,
        max_tokens=500
    )

    validation = validation_response.choices[0].message.content.strip()

    return {
        **state,
        "validation": validation
    }

# Construção do grafo
builder = StateGraph(ChatState)

# Adiciona os nós
builder.add_node("retriever", retrieve_context)
builder.add_node("chat", generate_answer)
builder.add_node("validator", AnswerValidatorAgent)

# Conexões entre os nós
builder.set_entry_point("retriever")
builder.add_edge("retriever", "chat")
builder.add_edge("chat", "validator")
builder.add_edge("validator", END)

# Finaliza e compila o grafo
graph = builder.compile()


In [6]:
#Iteração com o chat
while True:
    user_input = input("Você: ")
    if user_input.lower() in ["sair", "exit", "quit"]:
        break

    result = graph.invoke({"input": user_input})
    print("Bot:", result["answer"])
    print("Validação:", result["validation"])



Você:  sair
